# Intro/Imports

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import joblib
import pathlib as path
import copy

# Data Read-in

In [24]:
rawData = pd.read_csv('STEMVisualsSynthData.csv', header=0)
#remove unneeded column
rawData.drop('Index_within_Experiment', axis = 1, inplace = True)
#X is inputs--the three Concentrations, F_in, I0 (light intensity), and c_N_in (6)
X = rawData[['Time', 'C_X', 'C_N', 'C_L', 'F_in', 'C_N_in', 'I0']]
Y = X.copy(deep=True)
#drop unnecessary rows in Y
Y.drop('F_in', axis = 1, inplace = True)
Y.drop('C_N_in', axis = 1, inplace = True)
Y.drop('I0', axis = 1, inplace = True)
Y.drop('Time', axis = 1, inplace = True)
#Y vals should be X concentrations one timestep ahead, so remove the first index
Y.drop(index=0, inplace=True)
#To keep the two consistent, remove the last index of X
X.drop(index=19999, inplace=True)
#separate the times out into their own little thing for later use

C:\Users\kenda\AppData\Local\Temp\ipykernel_27052\3039068683.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(index=19999, inplace=True)


# Train/Val/Test Split

In [27]:
#create training/validate/testing splits
train_ratio = 0.60
validation_ratio = 0.10
test_ratio = 0.20

# train is now 75% of the entire data set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

#Separate Time out--we don't want this as a feature the model learns (since it's already incorporated in by how X and Y are structured!)
#however, having the time values will be useful for plotting later
XTrainTime = X_train.pop('Time')
XValTime = X_val.pop('Time')
XTestTime = X_test.pop('Time')

# Data Preprocessing

In [26]:
stScalerX = preprocessing.StandardScaler().fit(X_train['C_L', 'F_in', 'I0', 'C_N_in'])
stScalerY = preprocessing.StandardScaler().fit(Y_train['C_L'])

pTransformX = preprocessing.PowerTransformer().fit(X_train['C_X', 'C_N'])
pTransformY = preprocessing.PowerTransformer().fit(Y_train['C_X', 'C_N'])


ValueError: Expected 2D array, got 1D array instead:
array=[7.450000e-05 1.481650e-04 2.208390e-04 ... 6.985804e-03 6.986038e-03
 6.986194e-03].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.